In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

In [11]:
df_train = pd.read_csv('data/train.csv')

In [12]:
df_train.head

<bound method NDFrame.head of        label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0          1       0       0       0       0       0       0       0       0   
1          0       0       0       0       0       0       0       0       0   
2          1       0       0       0       0       0       0       0       0   
3          4       0       0       0       0       0       0       0       0   
4          0       0       0       0       0       0       0       0       0   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
41995      0       0       0       0       0       0       0       0       0   
41996      1       0       0       0       0       0       0       0       0   
41997      7       0       0       0       0       0       0       0       0   
41998      6       0       0       0       0       0       0       0       0   
41999      9       0       0       0       0       0       0       0       0   

       pi

In [ ]:
#AJ

In [ ]:
#AJ

In [ ]:
#AJ

In [ ]:
#AJ

In [ ]:
#AJ

In [ ]:
#AJ

In [ ]:
#EB

In [ ]:
#EB

In [ ]:
#EB

In [ ]:
#EB

In [ ]:
#EB